In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from get_dataframes import GetDataframes
import hjson

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)

In [2]:
datas = GetDataframes(config)
link = "actors_movies"
df = datas.get_dataframes(
    link,
)

2023-10-30 10:39:36 INFO     Parquet loaded ! Importing actors_movies...
2023-10-30 10:39:37 INFO     Dataframe actors_movies ready to use!


**JME SUIS ARRETÉ LA**


In [4]:
df = df.copy()

In [5]:
df.head()

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,status,region,cuts,person_id,person_name,person_birthdate,person_job,person_role,person_film,person_index
0,tt0020403,Show of Shows,1929,128,[Comedy],5.80,462,en,Show of Shows,1.12,...,Released,FR,1920-1929,nm0183823,William Courtenay,1875,actor,[The Minister - Guillotine Sequence],"[tt0020403, tt0020355, tt0000009, tt0021535]",2
1,tt0020403,Show of Shows,1929,128,[Comedy],5.80,462,en,Show of Shows,1.12,...,Released,FR,1920-1929,nm0098376,Hobart Bosworth,1867,actor,[Executioner - Guillotine Sequence],"[tt0006280, tt0003731, tt0003362, tt0015624]",4
2,tt0020403,Show of Shows,1929,128,[Comedy],5.80,462,en,Show of Shows,1.12,...,Released,FR,1920-1929,nm0912478,H.B. Warner,1876,actor,[The Victim - Guillotine Sequence],"[tt0027996, tt0030993, tt0029162, tt0038650]",3
3,tt0020403,Show of Shows,1929,128,[Comedy],5.80,462,en,Show of Shows,1.12,...,Released,FR,1920-1929,nm0269567,Frank Fay,1891,actor,[Master of Ceremonies],"[tt0022902, tt0021132, tt0029322, tt0021917]",1
4,tt0165435,The Arrival from the Darkness,1921,60,"[Fantasy, Horror]",6.40,109,cs,Příchozí z temnot,0.60,...,Released,FR,1920-1929,nm0841389,Josef Sváb-Malostranský,1860,actor,[Jan],"[tt0000190, tt0006769, tt0310395, tt1075008]",3


In [6]:
df.person_role.unique

<bound method Series.unique of 0         [The Minister - Guillotine Sequence]
1          [Executioner - Guillotine Sequence]
2           [The Victim - Guillotine Sequence]
3                       [Master of Ceremonies]
4                                        [Jan]
                          ...                 
212706                               [Unknown]
212707                                  [Self]
212708                                  [Self]
212709                                  [Self]
212710                                  [Self]
Name: person_role, Length: 212711, dtype: object>

In [8]:
df["titre_id"].unique().size

52439

In [9]:
condi2 = (
    (df["titre_id"].str.contains("tt0068991")) &
    (df["person_id"].str.contains("nm0000367"))
)
df[condi2]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,status,region,cuts,person_id,person_name,person_birthdate,person_job,person_role,person_film,person_index
52076,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,Released,FR,1970-1979,nm0000367,Gérard Depardieu,1948,actor,[Salesman],"[tt0103594, tt0099699, tt0099334, tt0120744]",3


In [10]:
len(df[df["person_name"].str.contains("Gérard Depardieu")])

135